<h1>Basic Elastic Test!</h1>

In [1]:
!pip3 install requests

In [2]:
import requests
import json 
#MY ELASTICSEARCH URL
URL = "http://127.0.0.1:9200/"

In [3]:
#Request URL
R_URL = URL+"shakespeare/_doc/38249"

In [4]:
# sending post request and saving response as response object 
r = requests.get(url = R_URL) 
  
# extracting response text  
response = r.text 
print("The response is:%s"%response) 

The response is:{"_index":"shakespeare","_type":"_doc","_id":"38249","_version":1,"_seq_no":37967,"_primary_term":1,"found":true,"_source":{"line_id":38250,"play_name":"Henry V","speech_number":26,"line_number":"3.6.96","speaker":"FLUELLEN","text_entry":"prave man."}}


In [5]:
y = json.loads(response)

In [6]:
# the result is a Python dictionary:
print(y["_source"]["play_name"]) 

Henry V


In [7]:
print(y)

{'_index': 'shakespeare', '_type': '_doc', '_id': '38249', '_version': 1, '_seq_no': 37967, '_primary_term': 1, 'found': True, '_source': {'line_id': 38250, 'play_name': 'Henry V', 'speech_number': 26, 'line_number': '3.6.96', 'speaker': 'FLUELLEN', 'text_entry': 'prave man.'}}


In [8]:
# pretty printing data
pretty_data = json.dumps(y, indent=4)
 
print(pretty_data)

{
    "_index": "shakespeare",
    "_type": "_doc",
    "_id": "38249",
    "_version": 1,
    "_seq_no": 37967,
    "_primary_term": 1,
    "found": true,
    "_source": {
        "line_id": 38250,
        "play_name": "Henry V",
        "speech_number": 26,
        "line_number": "3.6.96",
        "speaker": "FLUELLEN",
        "text_entry": "prave man."
    }
}


<h1>Let's try a search request</h1>

In [9]:
#Request URL
R_URL = URL+"shakespeare/_search"

In [10]:
#PAYLOAD
R_data ="{\"query\": { \"match\" : { \"text_entry\" : {\"query\" : \"knife war edge\",\"operator\" : \"and\"}}} }"
JSON_DATA = y = json.loads(R_data)

In [11]:
response = requests.post(url = R_URL, json = JSON_DATA)

In [12]:
response  = response.text 
print("The response is:%s"%response) 

The response is:{"took":8,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":1,"relation":"eq"},"max_score":20.073416,"hits":[{"_index":"shakespeare","_type":"_doc","_id":"19","_score":20.073416,"_source":{"line_id":20,"play_name":"Henry IV","speech_number":1,"line_number":"1.1.17","speaker":"KING HENRY IV","text_entry":"The edge of war, like an ill-sheathed knife,"}}]}}


In [13]:
#Convert to JSON
y = json.loads(response)
print(y)
# pretty printing data
pretty_data = json.dumps(y, indent=4)
 
print(pretty_data)

{'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 20.073416, 'hits': [{'_index': 'shakespeare', '_type': '_doc', '_id': '19', '_score': 20.073416, '_source': {'line_id': 20, 'play_name': 'Henry IV', 'speech_number': 1, 'line_number': '1.1.17', 'speaker': 'KING HENRY IV', 'text_entry': 'The edge of war, like an ill-sheathed knife,'}}]}}
{
    "took": 8,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 1,
            "relation": "eq"
        },
        "max_score": 20.073416,
        "hits": [
            {
                "_index": "shakespeare",
                "_type": "_doc",
                "_id": "19",
                "_score": 20.073416,
                "_source": {
                    "line_id": 20,
                    "play_name"

In [14]:
#Print the specific field
print(y["hits"]["hits"][0]["_source"]["text_entry"])

The edge of war, like an ill-sheathed knife,


<h1>Lercio Crawling</h1>

In [15]:
#Define requests object
import requests
import json 
#MY ELASTICSEARCH URL
URL = "http://127.0.0.1:9200/"

In [16]:
#New Lercio Index
index = "{ \"mappings\":{\"properties\":{\"title\" : {\"type\": \"text\" }, \"url\" : {\"type\": \"keyword\" },  \"author\" : { \"type\" : \"text\" }, \"abstract\" : { \"type\" : \"text\" }}}}"
JSON_DATA = json.loads(index)

#Request URL
R_URL = URL+"lercio"
RINS_URL = URL+"lercio/_doc/"

response = requests.put(url = R_URL, json = JSON_DATA)
response  = response.text 

#Convert to JSON
y = json.loads(response)
# pretty printing data
pretty_data = json.dumps(y, indent=4)
print(pretty_data)

{
    "acknowledged": true,
    "shards_acknowledged": true,
    "index": "lercio"
}


In [17]:
import sys
!pip3 install scrapy==2.6.2
import attrs

In [18]:
#!pip3 install pyopenssl==22.0.0

In [19]:
import scrapy

/Users/uniba/opt/anaconda3/envs/scrapy/lib/python3.6/site-packages/OpenSSL/crypto.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography import utils, x509


In [20]:
import scrapy
from scrapy.selector import Selector
from scrapy.crawler import CrawlerProcess

class LercioScraper(scrapy.Spider):
    name = "LercioPoliticalStories"
    start_urls = [
        'https://www.lercio.it/category/politica/',
    ]
    def parse(self, response):
        next_page = response.css('a.next')[0].attrib['href']
        next_page_num = int(next_page.split('/')[-2])
        if next_page is not None and next_page_num <30:
           # print(next_page_num)
            yield scrapy.Request(next_page, callback=self.parse, dont_filter=True)
            
        
        #Select the news containers
        newses = response.css('.articleTextBox').getall()
        for container in enumerate(newses):
            #print(container[1])
            #Get Title, Date, and Abstract
            news = Selector(text=container[1]).xpath('//div/a')
            #print(news)
            n_link = news[-1].attrib['href']
            #print(n_link)
            
            title = Selector(text=Selector(text=container[1]).css('.articleTitle')[0].get()).xpath('//text()').get().replace("\""," ").replace("'"," ")
            #print(title)
            autore = Selector(text=Selector(text=container[1]).css('.EQarticleAuthor')[0].get()).xpath('//text()')[1].get().replace("\""," ").replace("'"," ")
            #print(autore)
            abstract = Selector(text=Selector(text=container[1]).css('.articleText').get()).xpath('//text()')[0].get().replace("\""," ").replace("'"," ")
            print(abstract)
            
            #Create json to send
            document = "{\"title\":\""+title+"\", \"url\":\""+n_link+"\",\"author\":\""+autore+"\",\"abstract\":\""+abstract+"\" }"
            JSON_DATA = json.loads(document)
            print(JSON_DATA)
            response = requests.post(url = RINS_URL, json = JSON_DATA)
            response  = response.text 

            #Convert to JSON
            y = json.loads(response)
            # pretty printing data
            pretty_data = json.dumps(y, indent=4)
            #print(pretty_data)
           
process = CrawlerProcess()
process.crawl(LercioScraper)
process.start()

2022-12-12 18:27:22 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-12-12 18:27:22 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.4.0, Python 3.6.13 |Anaconda, Inc.| (default, Feb 23 2021, 12:58:59) - [GCC Clang 10.0.0 ], pyOpenSSL 22.0.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform Darwin-21.1.0-x86_64-i386-64bit
2022-12-12 18:27:22 [scrapy.crawler] INFO: Overridden settings:
{}
2022-12-12 18:27:22 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-12-12 18:27:22 [scrapy.extensions.telnet] INFO: Telnet Password: 1264998751eee6b7
2022-12-12 18:27:22 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2022-12-12 18:27:23 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scra

 Napoli – Dopo le recenti uscite di Alessandro Orsini sull’invas... sull’operazione speciale in corso in Ucraina, un gruppo…
{'title': 'Ritrovato appunto di Orsini sui Mondiali ‘82:  Contro il Brasile l’Italia deve arrendersi subito”', 'url': 'https://www.lercio.it/ritrovato-appunto-di-orsini-sui-mondiali-82-contro-il-brasile-litalia-deve-arrendersi-subito/', 'author': 'Augusto Rasori', 'abstract': ' Napoli – Dopo le recenti uscite di Alessandro Orsini sull’invas... sull’operazione speciale in corso in Ucraina, un gruppo…'}
 ROMA – Sono ore febbrili quelle che si stanno vivendo al palazzo del Viminale, dove Matteo Salvini si…
{'title': 'Salvini:  Mia figlia ha quasi finito di disegnare il progetto per il Ponte sullo Stretto ', 'url': 'https://www.lercio.it/salvini-mia-figlia-ha-quasi-finito-di-disegnare-il-progetto-per-il-ponte-sullo-stretto/', 'author': 'Augusto Rasori', 'abstract': ' ROMA – Sono ore febbrili quelle che si stanno vivendo al palazzo del Viminale, dove Matteo Salvini si

2022-12-12 18:27:23 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 162
2022-12-12 18:27:23 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:23 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 163
2022-12-12 18:27:23 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:23 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 163
2022-12-12 18:27:23 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:23 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 164
2022-12-12 18:27:23 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200


 Fabio Corigliano
{'title': 'Vaticano. Spunta un audio choc del Papa:  Forse sarebbe giusto iniziare a pagare l IMU ', 'url': 'https://www.lercio.it/vaticano-spunta-un-audio-choc-del-papa-forse-sarebbe-giusto-iniziare-a-pagare-limu/', 'author': 'Fabio Corigliano', 'abstract': ' Fabio Corigliano'}
 Andrea Sesta
{'title': 'Legge di Bilancio, il Governo introduce i numeri immaginari per far quadrare i conti', 'url': 'https://www.lercio.it/legge-di-bilancio-il-governo-introduce-i-numeri-immaginari-per-far-quadrare-i-conti/', 'author': 'Andrea H. Sesta', 'abstract': ' Andrea Sesta'}
 Francesco Conte
{'title': ' Evviva l umiliazione , il Ministro Valditara esulta per le migliaia di insulti ricevuti', 'url': 'https://www.lercio.it/evviva-lumiliazione-il-ministro-valditara-esulta-per-le-migliaia-di-insulti-ricevuti/', 'author': 'Il Democritico', 'abstract': ' Francesco Conte'}
 Andrea Sesta
{'title': '\xa0Bonus Matrimoni, controproposta di Forza Italia:  20mila euro a chi sposa Silvio Berlusco

2022-12-12 18:27:24 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 164
2022-12-12 18:27:24 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:24 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 163


...

 Roma (DPCM) – Si stanno finalmente districando i nodi che da settimane rendevano inintelligibile il comportamento da seguire…
{'title': 'Governo:  Natale solo con gli affetti più cari , Salvini a pranzo con il suo commercialista', 'url': 'https://www.lercio.it/conte-natale-solo-con-gli-affetti-piu-cari-salvini-a-pranzo-con-il-suo-commercialista/', 'author': 'Augusto Rasori', 'abstract': ' Roma (DPCM) – Si stanno finalmente districando i nodi che da settimane rendevano inintelligibile il comportamento da seguire…'}
 Patrizio Smiraglia / Stefano Pisani
{'title': 'DPCM di Natale, Conte chiarisce:  Trenini di Capodanno un posto sì e uno no ', 'url': 'https://www.lercio.it/dpcm-di-natale-conte-chiarisce-trenini-di-capodanno-un-posto-si-e-uno-no/', 'author': 'lercio', 'abstract': ' Patrizio Smiraglia / Stefano Pisani'}
 Andrea Bonechi
{'title': 'Governo, gli esperti temono una seconda ondata di renzismo', 'url': 'https://www.lercio.it/governo-gli-esperti-temono-una-seconda-ondata-di-renzism

2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 165
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 165
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 165
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 165


 ROMA – Con l’imminente arrivo del vaccino è sempre più spasmodica l’attesa di un piano di gestione, da…
{'title': 'Vaccino, governo lancia il Click-day:  Bonus di 1000 anticorpi ai primi che si prenotano ', 'url': 'https://www.lercio.it/vaccino-governo-lancia-il-click-day-bonus-di-1000-anticorpi-ai-primi-che-si-prenotano/', 'author': 'Vittorio Lattanzi', 'abstract': ' ROMA – Con l’imminente arrivo del vaccino è sempre più spasmodica l’attesa di un piano di gestione, da…'}
 Sergio Marinelli
{'title': ' Aveva dei sentimenti . Espulso dirigente di Confindustria', 'url': 'https://www.lercio.it/aveva-dei-sentimenti-espulso-dirigente-di-confindustria/', 'author': 'Sergio Marinelli', 'abstract': ' Sergio Marinelli'}
 Roma – Il ministro della Salute Roberto Speranza ha annunciato oggi che il governo utilizzerà i giocatori di…
{'title': 'Piano vaccinazioni, governo assume 10.000 campioni di freccette', 'url': 'https://www.lercio.it/piano-vaccinazioni-governo-assume-campioni-di-freccette/', 'au

2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 165
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 166
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:27:51 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_doc/ HTTP/1.1" 201 165
2022-12-12 18:27:51 [scrapy.core.engine] INFO: Closing spider (finished)
2022-12-12 18:27:51 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 8551,
 'downloader/request_count': 29,
 'downloader/request_method_count/GET': 29,
 'downloader/response_bytes': 1902038,
 'downloader/response_count': 29,
 'downloader/response_status_c

 Andrea Bonechi
{'title': 'Rimpasto di Governo, Renzi:  Come nuovo Premier avrei giusto in mente un nome ', 'url': 'https://www.lercio.it/rimpasto-di-governo-renzi-come-nuovo-premier-avrei-giusto-in-mente-un-nome/', 'author': 'Chiorbaciov', 'abstract': ' Andrea Bonechi'}
 Augusto Rasori
{'title': 'Camera, focolaio di Covid passa al Gruppo Misto', 'url': 'https://www.lercio.it/camera-focolaio-di-covid-passa-al-gruppo-misto/', 'author': 'Augusto Rasori', 'abstract': ' Augusto Rasori'}


In [21]:
#Let's do a search
#Request URL
R_URL = URL+"lercio/_search"

In [28]:
#PAYLOAD
R_data ="{\"query\": { \"match\" : { \"title\" : {\"query\" : \"Salvini benzina\",\"operator\" : \"and\"}}} }"
JSON_DATA = y = json.loads(R_data)

In [30]:
response = requests.post(url = R_URL, json = JSON_DATA)
response  = response.text 

#Convert to JSON
y = json.loads(response)
# pretty printing data
pretty_data = json.dumps(y, indent=4)
print(pretty_data)

2022-12-12 18:29:22 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): 127.0.0.1:9200
2022-12-12 18:29:22 [urllib3.connectionpool] DEBUG: http://127.0.0.1:9200 "POST /lercio/_search HTTP/1.1" 200 445


{
    "took": 1,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 1,
            "relation": "eq"
        },
        "max_score": 6.6290607,
        "hits": [
            {
                "_index": "lercio",
                "_type": "_doc",
                "_id": "7RlgB4UBBgQuFi8ViLtc",
                "_score": 6.6290607,
                "_source": {
                    "title": "Salvini:  Appena finisco di togliere le accise sulla benzina inizio a occuparmi del ponte sullo Stretto ",
                    "url": "https://www.lercio.it/salvini-appena-finisco-di-togliere-le-accise-sulla-benzina-inizio-a-occuparmi-del-ponte-sullo-stretto/",
                    "author": "Augusto Rasori",
                    "abstract": " Leggere la lista dei ministri si \u00e8 rivelato rassicurante, dato che chi aveva pessime aspettative non \u00e8 rimasto\u2026"
   